In [31]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [32]:
SEED=1

In [33]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [34]:
torch.cuda.device_count()

1

### Set the architecture to resnet 18 below

In [35]:
START_EPOCH = 0

In [36]:
##########################
ARCH = 'resnet18'# set the architecture to RESNET 18
# please look up how to do that
########################
EPOCHS = 200
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
PRINT_FREQ = 50
TRAIN_BATCH=128
VAL_BATCH=128
WORKERS=2
# TRAINDIR="/workspace/data/imagenet2012/train"
# VALDIR="/workspace/data/imagenet2012/val"

In [37]:
# TRAINDIR="/CINIC/train"
# VALDIR="/CINIC/valid"

### Check if cuda is available here

In [38]:
# check if cuda is available in this cell
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


### Assign your GPU below

In [39]:
# Assign your GPU in this cell
GPU = 0

In [40]:
# set your active device to your GPU in this cell
torch.cuda.set_device(GPU)

In [41]:
# enable algorithm optimization
cudnn.benchmark = True

### Fill in the heart of the train section below

In [42]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch model to train mode here
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        #####################
        # send the images to cuda device
        # send the target to cuda device
        if GPU is not None:
            images = images.cuda(GPU, non_blocking=True)
        if torch.cuda.is_available():
            target = target.cuda(GPU, non_blocking=True)


        # compute output - forward pass
        output = model(images)

        # compute loss 
        loss = criterion(output, target)


        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        # zero out gradients in the optimier - think about gradient accumulation
        optimizer.zero_grad()
        
        # backward pass - calculate gradients
        loss.backward()
        
        # update the weights
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

#### Fill in the validate section below

In [43]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            # send the images and target to cuda
            if GPU is not None:
                images = images.cuda(GPU, non_blocking=True)
            if torch.cuda.is_available():
                target = target.cuda(GPU, non_blocking=True)

            # compute output
            output = model(images)
            
            # compute loss
            loss  = criterion(output, target)


            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

### Save the checkpoint

In [44]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    # save the model state!
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [45]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [46]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [47]:
# if we are adjusting the LR manually use this
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [48]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [49]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [50]:
normalize = transforms.Normalize(mean=cifar_mean_RGB, std=cifar_std_RGB)

In [51]:
IMG_SIZE = 32
# IMG_SIZE = 224

In [52]:
NUM_CLASSES = 10

### Initialize the model using the architecture you selected above

In [53]:
# select the model
model = models.__dict__[ARCH]() 
model.fc = nn.Linear(model.fc.in_features, NUM_CLASSES)

### Send the model to the cuda device

In [54]:
# send the model to the cuda device
model.cuda(GPU)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Instantiate the loss to cross entropy

In [55]:
# use the cross-entropy loss
criterion = nn.CrossEntropyLoss().cuda(GPU)

### Instantiate the optimizer to SGD

In [57]:
# use SGD .. use the momentum and weight decay vars
optimizer = torch.optim.SGD(model.parameters(), LR,
                               momentum=MOMENTUM,
                               weight_decay=WEIGHT_DECAY)
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

#### Create the learning rate scheduler

In [58]:
# use CosineAnnealingLR
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [59]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cifar_mean_RGB, cifar_std_RGB),
])

### Create the train dataset object

In [61]:
# use torchvision.datasets.CIFAR10
train_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)

0it [00:00, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


In [62]:
transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar_mean_RGB, cifar_std_RGB),
])

### Create the val dataset object

In [63]:
# use torchvision.datasets.CIFAR10
val_dataset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_val)

Files already downloaded and verified


### Create the train dataloader

In [64]:
# fill this in
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=TRAIN_BATCH, shuffle=True,
    num_workers=WORKERS, pin_memory=True, sampler=None)

### Create the c

In [65]:
# fill this in..
val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=VAL_BATCH, shuffle=False,
    num_workers=WORKERS, pin_memory=True, sampler=None) 

In [66]:
best_acc1 = 0

In [67]:
for epoch in range(START_EPOCH, EPOCHS):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

Epoch: [0][  0/391]	Time 11.263 (11.263)	Data  0.386 ( 0.386)	Loss 2.5604e+00 (2.5604e+00)	Acc@1  11.72 ( 11.72)	Acc@5  42.19 ( 42.19)
Epoch: [0][ 50/391]	Time  0.100 ( 0.323)	Data  0.001 ( 0.008)	Loss 2.0755e+00 (2.9714e+00)	Acc@1  24.22 ( 17.37)	Acc@5  85.94 ( 68.69)
Epoch: [0][100/391]	Time  0.104 ( 0.214)	Data  0.000 ( 0.005)	Loss 1.9098e+00 (2.5650e+00)	Acc@1  25.00 ( 21.83)	Acc@5  81.25 ( 74.74)
Epoch: [0][150/391]	Time  0.108 ( 0.177)	Data  0.001 ( 0.004)	Loss 2.0400e+00 (2.3774e+00)	Acc@1  32.81 ( 24.47)	Acc@5  89.84 ( 77.90)
Epoch: [0][200/391]	Time  0.100 ( 0.158)	Data  0.001 ( 0.003)	Loss 1.7588e+00 (2.2476e+00)	Acc@1  35.94 ( 26.82)	Acc@5  90.62 ( 80.10)
Epoch: [0][250/391]	Time  0.099 ( 0.147)	Data  0.000 ( 0.003)	Loss 1.7241e+00 (2.1487e+00)	Acc@1  43.75 ( 28.99)	Acc@5  91.41 ( 81.74)
Epoch: [0][300/391]	Time  0.101 ( 0.139)	Data  0.001 ( 0.002)	Loss 1.6643e+00 (2.0725e+00)	Acc@1  37.50 ( 30.60)	Acc@5  90.62 ( 82.93)
Epoch: [0][350/391]	Time  0.102 ( 0.134)	Data  0.001 ( 

Exception in thread Thread-141:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/_utils/pin_memory.py", line 25, in _pin_memory_loop
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 113, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/reductions.py", line 282, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
  File "/usr/lib/pyth

KeyboardInterrupt: 